# 0. 패키지 로드

In [1]:
import os
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import QuantileRegressor
import matplotlib.pyplot as plt
from matplotlib.ticker import MultipleLocator
import seaborn as sns
import pickle
import warnings
warnings.filterwarnings("ignore", category=UserWarning, module='openpyxl')

In [2]:
def uv_to_wsd(u_wind_speed, v_wind_speed):

    u_ws = u_wind_speed.to_numpy()
    v_ws = v_wind_speed.to_numpy()

    wind_speed = np.nansum([u_ws**2, v_ws**2], axis=0)**(1/2.)

    wind_direction = np.rad2deg(np.arctan2(v_ws, u_ws+1e-6))
    wind_direction[wind_direction < 0] += 360

    wind_direction = 270 - wind_direction
    wind_direction[wind_direction < 0] += 360

    return wind_speed, wind_direction

In [5]:
'''
path 설정 !!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
'''
path =

# 1. 영광

## 1-1 20-22년도 발전량 학습

### (1) feature 데이터 불러오기 및 변수 선택

In [48]:
# 20-22년도 데이터
df_feature = pd.read_pickle(os.path.join(path, "1-3. train_ldaps_yeonggwang.pkl"))

# wind_speed 변수 추출
df_feature["wind_speed"], df_feature["wind_direction"] = uv_to_wsd(
    df_feature["wind_u_10m"], df_feature["wind_v_10m"])

df_feature["storm_speed"], df_feature["storm_direction"] = uv_to_wsd(
    df_feature["storm_u_5m"], df_feature["storm_v_5m"])



# 변수 선택
df_x = df_feature.loc[:,['specific_humid', 'temp_air', 'wind_speed', 'wind_u_10m', 'storm_u_5m', 'turbine_id']]

# 월, 시간 변수 추출
df_x['month'] = df_x.index.month.astype('category')
df_x['hour'] = df_x.index.hour.astype('category')

### (2) target 데이터 불러오기

In [50]:
# 20-22년도 발전량
train_y = pd.read_csv(os.path.join(path, "1-1. train_y.csv"))
train_y.end_datetime = pd.to_datetime(
    train_y.end_datetime
).dt.tz_convert("Asia/Seoul")

yg_y = train_y[train_y.plant_name == "영광풍력"]
true_y = yg_y.copy()[["end_datetime", "energy_kwh"]]

In [17]:
# 수치형 변수 시간별 평균
wind_mean = df_x.groupby(df_x.index)['wind_speed'].mean().to_frame(name='wind_mean')
temp_mean = df_x.groupby(df_x.index)['temp_air'].mean().to_frame(name='temp_mean')
humi_mean = df_x.groupby(df_x.index)['specific_humid'].mean().to_frame(name='humi_mean')
uwin_mean = df_x.groupby(df_x.index)['wind_u_10m'].mean().to_frame(name='uwin_mean')
usto_mean = df_x.groupby(df_x.index)['storm_u_5m'].mean().to_frame(name='usto_mean')

In [19]:
# 시간별 평균값으로 구성된 데이터프레임
ew = yg_y.merge(
    temp_mean.reset_index(),
    left_on="end_datetime",
    right_on="dt"
).merge(
    humi_mean.reset_index(),
    on="dt"
).merge(
    wind_mean.reset_index(),
    on="dt"
).merge(
    uwin_mean.reset_index(),
    on="dt"
).merge(
    usto_mean.reset_index(),
    on="dt"
)[["dt", "energy_kwh", "temp_mean", "humi_mean", "wind_mean", "uwin_mean", "usto_mean"]]
ew['month'] = ew['dt'].dt.month.astype('category')
ew['hour'] = ew['dt'].dt.hour.astype('category')

### (3) 스케일링 및 학습

In [ ]:
# 데이터 준비
X = ew[["temp_mean", "humi_mean", "wind_mean", "uwin_mean", "usto_mean"]].values
y = ew['energy_kwh'].values

scaler_X = StandardScaler()
X_scaled = scaler_X.fit_transform(X)  # 독립변수 표준화

# 4차 다항 변환
poly = PolynomialFeatures(degree=4)
X_poly = poly.fit_transform(X_scaled)

# 발전량 학습
model = QuantileRegressor(quantile=0.5, alpha=0, solver='highs')
model.fit(X_poly, y)

### (4) 예측

In [ ]:
ls_pred = []

# 터빈별로 데이터를 나누어 20-22년도 발전량 예측
for turb in df_x['turbine_id'].unique():
    # 해당 터빈의 데이터 선택
    cur_x = df_x[df_x['turbine_id'] == turb][['temp_air', 'specific_humid', 'wind_speed', 'wind_u_10m', 'storm_u_5m']].copy().values

    X_scaled = scaler_X.transform(cur_x)
    X_new_poly = poly.transform(X_scaled)

    # 예측 수행
    cur_pred_y = model.predict(X_new_poly) / 35

    # 예측 결과를 데이터프레임으로 변환하고, 터빈 ID 추가
    cur_pred_df = pd.DataFrame({
        'predicted_energy_kwh': cur_pred_y,
        'turbine_id': turb
    })

    # 리스트에 추가
    ls_pred.append(cur_pred_df)

# 최종 데이터프레임으로 결합
df_pred_y = pd.concat(ls_pred, axis=0)
pred_y = df_pred_y.groupby(df_pred_y.index)['predicted_energy_kwh'].sum().to_frame()
pred_y = np.maximum(0, pred_y)

In [ ]:
# pred_y 인덱스 조정
df_x = df_x[df_x['turbine_id']=='WTG01']
df_x = df_x.reset_index()
pred_y['dt'] = df_x['dt']
pred_y = pred_y.set_index('dt')

# 실제발전량, 예측발전량 데이터프레임
result = true_y.merge(
    pred_y.reset_index(),
    left_on="end_datetime",
    right_on="dt"
)[["dt", "energy_kwh", "predicted_energy_kwh"]]

### (5) 2020~2022년 feature + target 데이터 저장 → train_yg로 사용될 예정

In [52]:
# 23년도 피처 & 예측발전량 합친 데이터프레임
df_feature_numeric = df_feature.select_dtypes(include='number')
df = df_feature_numeric.groupby(df_feature.index).mean()
true_y_dict = true_y.set_index('end_datetime')['energy_kwh'].to_dict()
df['energy_kwh'] = df.index.map(true_y_dict)

In [ ]:
df.to_csv(os.path.join(path,"영광종합.csv"), encoding='cp949')

## 1-2 23년도 예측

### (1) feature 데이터 불러오기 및 변수 선택

In [ ]:
df_feature = pd.read_pickle(os.path.join(path, "1-3. test_ldaps_yeonggwang.pkl"))

df_feature["wind_speed"], df_feature["wind_direction"] = uv_to_wsd(
    df_feature["wind_u_10m"], df_feature["wind_v_10m"])
df_feature["storm_speed"], df_feature["storm_direction"] = uv_to_wsd(
    df_feature["storm_u_5m"], df_feature["storm_v_5m"])

df_x = df_feature.loc[:,['specific_humid', 'temp_air', 'wind_speed', 'wind_u_10m', 'storm_u_5m', 'turbine_id']]

df_x['month'] = df_x.index.month.astype('category')
df_x['hour'] = df_x.index.hour.astype('category')

### (2) 예측

In [ ]:
ls_pred = []

# 터빈별로 데이터를 나누어 예측
for turb in df_x['turbine_id'].unique():
    # 해당 터빈의 데이터 선택
    cur_x = df_x[df_x['turbine_id'] == turb][['temp_air', 'specific_humid', 'wind_speed', 'wind_u_10m', 'storm_u_5m']].copy().values

    X_scaled = scaler_X.transform(cur_x)
    X_new_poly = poly.transform(X_scaled)

    # 예측 수행
    cur_pred_y = model.predict(X_new_poly) / 35

    # 예측 결과를 데이터프레임으로 변환하고, 터빈 ID 추가
    cur_pred_df = pd.DataFrame({
        'predicted_energy_kwh': cur_pred_y,
        'turbine_id': turb
    })

    # 리스트에 추가
    ls_pred.append(cur_pred_df)

# 최종 데이터프레임으로 결합
df_pred_y = pd.concat(ls_pred, axis=0)
pred_y = df_pred_y.groupby(df_pred_y.index)['predicted_energy_kwh'].sum().to_frame()
pred_y = np.maximum(0, pred_y)

### (3) 2023년 실제 기상 데이터 기반 2023년 예측 발전량(target) 데이터 저장 → test_yg로 사용될 예정

In [ ]:
# pred_y 인덱스 조정
df_x = df_x[df_x['turbine_id']=='WTG01']
df_x = df_x.reset_index()  # 기존 인덱스를 칼럼으로 이동 ('dt'라는 이름으로 들어감)
pred_y['dt'] = df_x['dt']
pred_y = pred_y.set_index('dt')

In [ ]:
pred_y.to_csv(os.path.join(path, "영광23.csv"), encoding='cp949', index=False)

# 2. 경주

## 2-1 20-22년도 발전량 학습

### (1) feature 데이터 불러오기 및 변수 선택

In [40]:
# 20-22년도 데이터
df_feature = pd.read_pickle(os.path.join(path, "1-4. train_ldaps_gyeongju.pkl"))

# wind_speed 변수 추출
df_feature["wind_speed"], df_feature["wind_direction"] = uv_to_wsd(
    df_feature["wind_u_10m"], df_feature["wind_v_10m"])
df_feature["storm_speed"], df_feature["storm_direction"] = uv_to_wsd(
    df_feature["storm_u_5m"], df_feature["storm_v_5m"])

# 변수 선택
df_x = df_feature.loc[:,['specific_humid', 'temp_air', 'wind_speed', 'wind_u_10m', 'storm_u_5m', 'turbine_id']]

# 월, 시간 변수 추출
df_x['month'] = df_x.index.month.astype('category')
df_x['hour'] = df_x.index.hour.astype('category')

### (2) target 데이터 불러오기

In [42]:
# 20-22년도 발전량
train_y = pd.read_csv(os.path.join(path, "1-1. train_y.csv"))
train_y.end_datetime = pd.to_datetime(
    train_y.end_datetime
).dt.tz_convert("Asia/Seoul")

gj_y = train_y[train_y.plant_name == "경주풍력"]
true_y = gj_y.copy()[["end_datetime", "energy_kwh"]]

In [ ]:
# 수치형 변수 시간별 평균
wind_mean = df_x.groupby(df_x.index)['wind_speed'].mean().to_frame(name='wind_mean')
temp_mean = df_x.groupby(df_x.index)['temp_air'].mean().to_frame(name='temp_mean')
humi_mean = df_x.groupby(df_x.index)['specific_humid'].mean().to_frame(name='humi_mean')
uwin_mean = df_x.groupby(df_x.index)['wind_u_10m'].mean().to_frame(name='uwin_mean')
usto_mean = df_x.groupby(df_x.index)['storm_u_5m'].mean().to_frame(name='usto_mean')

In [ ]:
# 시간별 평균값으로 구성된 데이터프레임
ew = gj_y.merge(
    temp_mean.reset_index(),
    left_on="end_datetime",
    right_on="dt"
).merge(
    humi_mean.reset_index(),
    on="dt"
).merge(
    wind_mean.reset_index(),
    on="dt"
).merge(
    uwin_mean.reset_index(),
    on="dt"
).merge(
    usto_mean.reset_index(),
    on="dt"
)[["dt", "energy_kwh", "temp_mean", "humi_mean", "wind_mean", "uwin_mean", "usto_mean"]]
ew['month'] = ew['dt'].dt.month.astype('category')
ew['hour'] = ew['dt'].dt.hour.astype('category')

### (3) 스케일링 및 학습

In [ ]:
# 데이터 준비
X = ew[["temp_mean", "humi_mean", "wind_mean", "uwin_mean", "usto_mean", "month", "hour"]].values
y = ew['energy_kwh'].values

scaler_X = StandardScaler()
X_scaled = scaler_X.fit_transform(X)  # 독립변수 표준화

# 4차 다항 변환
poly = PolynomialFeatures(degree=4)
X_poly = poly.fit_transform(X_scaled)

# 발전량 학습
model = QuantileRegressor(quantile=0.5, alpha=0, solver='highs')
model.fit(X_poly, y)

QuantileRegressor(alpha=0)

### (4) 예측

In [ ]:
ls_pred = []

# 터빈별로 데이터를 나누어 20-22년도 발전량 예측
for turb in df_x['turbine_id'].unique():
    # 해당 터빈의 데이터 선택
    cur_x = df_x[df_x['turbine_id'] == turb][['temp_air', 'specific_humid', 'wind_speed', 'wind_u_10m', 'storm_u_5m', 'month', 'hour']].copy().values

    X_scaled = scaler_X.transform(cur_x)
    X_new_poly = poly.transform(X_scaled)

    # 예측 수행
    cur_pred_y = model.predict(X_new_poly) / 9

    # 예측 결과를 데이터프레임으로 변환하고, 터빈 ID 추가
    cur_pred_df = pd.DataFrame({
        'predicted_energy_kwh': cur_pred_y,
        'turbine_id': turb
    })

    # 리스트에 추가
    ls_pred.append(cur_pred_df)

# 최종 데이터프레임으로 결합
df_pred_y = pd.concat(ls_pred, axis=0)
pred_y = df_pred_y.groupby(df_pred_y.index)['predicted_energy_kwh'].sum().to_frame()
pred_y = np.maximum(0, pred_y)

In [ ]:
# pred_y 인덱스 조정
df_x = df_x[df_x['turbine_id']=='WTG01']
df_x = df_x.reset_index()
pred_y['dt'] = df_x['dt']
pred_y = pred_y.set_index('dt')

# 실제발전량, 예측발전량 데이터프레임
result = true_y.merge(
    pred_y.reset_index(),
    left_on="end_datetime",
    right_on="dt"
)[["dt", "energy_kwh", "predicted_energy_kwh"]]

### (5) 2020~2022년 feature + target 데이터 저장 → train_gj로 사용될 예정

In [44]:
# 23년도 피처 & 예측발전량 합친 데이터프레임
df_feature_numeric = df_feature.select_dtypes(include='number')
df = df_feature_numeric.groupby(df_feature.index).mean()
true_y_dict = true_y.set_index('end_datetime')['energy_kwh'].to_dict()
df['energy_kwh'] = df.index.map(true_y_dict)

In [33]:
df.to_csv(os.path.join(path, "경주종합.csv"), encoding='cp949')

## 2-2 23년도 예측

### (1) feature 데이터 불러오기 및 변수 선택

In [ ]:
df_feature = pd.read_pickle(os.path.join(path, "1-4. test_ldaps_gyeongju.pkl"))

df_feature["wind_speed"], df_feature["wind_direction"] = uv_to_wsd(
    df_feature["wind_u_10m"], df_feature["wind_v_10m"])
df_feature["storm_speed"], df_feature["storm_direction"] = uv_to_wsd(
    df_feature["storm_u_5m"], df_feature["storm_v_5m"])

df_x = df_feature.loc[:,['specific_humid', 'temp_air', 'wind_speed', 'wind_u_10m', 'storm_u_5m', 'turbine_id']]

df_x['month'] = df_x.index.month.astype('category')
df_x['hour'] = df_x.index.hour.astype('category')

### (2) 예측

In [ ]:
ls_pred = []

# 터빈별로 데이터를 나누어 예측
for turb in df_x['turbine_id'].unique():
    # 해당 터빈의 데이터 선택
    cur_x = df_x[df_x['turbine_id'] == turb][['temp_air', 'specific_humid', 'wind_speed', 'wind_u_10m', 'storm_u_5m', 'month', 'hour']].copy().values

    X_scaled = scaler_X.transform(cur_x)
    X_new_poly = poly.transform(X_scaled)

    # 예측 수행
    cur_pred_y = model.predict(X_new_poly) / 9

    # 예측 결과를 데이터프레임으로 변환하고, 터빈 ID 추가
    cur_pred_df = pd.DataFrame({
        'predicted_energy_kwh': cur_pred_y,
        'turbine_id': turb
    })

    # 리스트에 추가
    ls_pred.append(cur_pred_df)

# 최종 데이터프레임으로 결합
df_pred_y = pd.concat(ls_pred, axis=0)
pred_y = df_pred_y.groupby(df_pred_y.index)['predicted_energy_kwh'].sum().to_frame()
pred_y = np.maximum(0, pred_y)

In [ ]:
# pred_y 인덱스 조정
df_x = df_x[df_x['turbine_id']=='WTG01']
df_x = df_x.reset_index()  # 기존 인덱스를 칼럼으로 이동 ('dt'라는 이름으로 들어감)
pred_y['dt'] = df_x['dt']
pred_y = pred_y.set_index('dt')

### (3) 2023년 실제 기상 데이터 기반 2023년 예측 발전량(target) 데이터 저장 → test_gj로 사용될 예정

In [ ]:
pred_y.to_csv(os.path.join(path, "경주23.csv"), encoding='cp949')

# 3. 2023년 발전량 예측 (실제 2023년 데이터 기반이 아닌 순수 예측)

In [66]:
from datetime import datetime, timedelta
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
import xgboost as xgb
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, r2_score

In [56]:
# ldaps
## 영광
train_yg = pd.read_csv(os.path.join(path, "영광종합.csv"), encoding='cp949')
train_yg.set_index('dt', inplace=True)
train_yg.index = pd.to_datetime(train_yg.index).tz_convert('Asia/Seoul')


test_yg_y = pd.read_csv(os.path.join(path, "영광23.csv"), encoding='cp949')
test_yg_y.set_index('dt', inplace=True)

test_yg = pd.read_pickle(os.path.join(path, "1-3. test_ldaps_yeonggwang.pkl"))
test_yg = test_yg.select_dtypes(include='number').groupby(test_yg.index).mean()
test_yg = test_yg.merge(test_yg_y, how='outer', on=test_yg.index)
test_yg.set_index('key_0', inplace=True)
test_yg.index.name = 'dt'
test_yg.rename(columns={'predicted_energy_kwh': 'energy_kwh'}, inplace=True)

test_yg["wind_speed"], test_yg["wind_direction"] = uv_to_wsd(
    test_yg["wind_u_10m"], test_yg["wind_v_10m"]
)
test_yg["storm_speed"], test_yg["storm_direction"] = uv_to_wsd(
    test_yg["storm_u_5m"], test_yg["storm_v_5m"]
)


## 경주
train_gj = pd.read_csv(os.path.join(path, "경주종합.csv"), encoding='cp949')
train_gj.set_index('dt', inplace=True)
train_gj.index = pd.to_datetime(train_gj.index).tz_convert('Asia/Seoul')

test_gj_y = pd.read_csv(os.path.join(path, "경주23.csv"), encoding='cp949')
test_gj_y.set_index('dt', inplace=True)

test_gj = pd.read_pickle(os.path.join(path, "1-4. test_ldaps_gyeongju.pkl"))
test_gj = test_gj.select_dtypes(include='number').groupby(test_gj.index).mean()
test_gj = test_gj.merge(test_gj_y, how='outer', on=test_gj.index)
test_gj.set_index('key_0', inplace=True)
test_gj.index.name = 'dt'
test_gj.rename(columns={'predicted_energy_kwh': 'energy_kwh'}, inplace=True)

test_gj["wind_speed"], test_gj["wind_direction"] = uv_to_wsd(
    test_gj["wind_u_10m"], test_gj["wind_v_10m"]
)
test_gj["storm_speed"], test_gj["storm_direction"] = uv_to_wsd(
    test_gj["storm_u_5m"], test_gj["storm_v_5m"]
)

## 3-1 데이터 전처리
- 시퀀스 생성

## 3-2 영광

### (1) test 데이터에 2022년 데이터 일부 붙이기
- lookback_days에 따라 2022년 12월 혹은 11월 데이터까지 사용해야할 수도 있음

In [70]:
# 영광
test_yg_added = pd.concat([train_yg[train_yg.index >= '2022-11-01 00:00:00+09:00'], test_yg])
test_yg_added.index = pd.to_datetime(test_yg_added.index)

### (2) lookback_days 정의 후 lookback 계산

In [73]:
# lookback_days : 며칠 전까지를 x시퀀스로 만들건지
lookback_days = 5

# 시간화
lookback = 24 * lookback_days

### (3) 컬럼 추가

In [76]:
# 영광
yg_offset = test_yg_added[test_yg_added.index == '2022-12-31 00:00:00+09:00'].index[0] - pd.Timedelta(hours=lookback)
test_yg_final = test_yg_added[test_yg_added.index >= yg_offset]
test_yg_final['month'] = test_yg_final.index.month.astype('category')
test_yg_final['hour'] = test_yg_final.index.hour.astype('category')

C:\Users\jinhe\AppData\Local\Temp\ipykernel_11952\3493249188.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_yg_final['month'] = test_yg_final.index.month.astype('category')
C:\Users\jinhe\AppData\Local\Temp\ipykernel_11952\3493249188.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_yg_final['hour'] = test_yg_final.index.hour.astype('category')


### (4) 시퀀스 생성

In [79]:
# 전날 15시까지의 데이터를 사용하여 다음날 23시까지 예측
def create_features(feature, target, lookback, forecast_horizon=32):
    X, y = [], []

    for i in range(lookback, len(feature) - forecast_horizon):
      current_time = feature.index[i]  # 현재 인덱스의 시간

      if current_time.hour == 15:  # 15시까지의 데이터를 사용하여 예측
          x_1 = feature.iloc[(i - lookback) : (i + 1)][['energy_kwh', 'wind_speed', 'wind_u_10m', 'storm_u_5m', 'temp_air', 'specific_humid']].values.flatten().reshape(1, -1)
          x_2 = np.array([feature[['month', 'hour']].values[i]])
          x_to = np.hstack([x_1, x_2])
          X.append(x_to.reshape(-1))  # 과거 (lookback)시간의 피처 데이터

          y_1 = feature.iloc[(i + 1) : (i + 1 + forecast_horizon)]['energy_kwh'].values
          y.append(y_1.flatten())  # 다음 32시간 동안의 예측 값

    return np.array(X), np.array(y)

### (5) 변수선택, 스케일링, 모델링

In [ ]:
yg_list = []

# 피처와 타겟 정의
train_features = train_yg[['energy_kwh', 'wind_speed', 'wind_u_10m', 'storm_u_5m', 'temp_air', 'specific_humid']]
train_target = train_yg[['energy_kwh']]

test_features = test_yg_final[['energy_kwh', 'wind_speed', 'wind_u_10m', 'storm_u_5m', 'temp_air', 'specific_humid']]
test_target = test_yg_final[['energy_kwh']]

# 데이터 스케일링
scaler_features = MinMaxScaler()
scaler_target = MinMaxScaler()

scaled_x_train = pd.DataFrame(scaler_features.fit_transform(train_features), columns = ['energy_kwh', 'wind_speed', 'wind_u_10m', 'storm_u_5m', 'temp_air', 'specific_humid'])  # train_features에 다차원 배열 입력
scaled_x_test = pd.DataFrame(scaler_features.transform(test_features) , columns = ['energy_kwh', 'wind_speed', 'wind_u_10m', 'storm_u_5m', 'temp_air', 'specific_humid'])       # test_features도 동일한 방식으로 변환

scaled_x_train.set_index(train_yg.index, inplace=True)
scaled_x_test.set_index(test_yg_final.index, inplace=True)

# 풍속 데이터에 대한 스케일링
scaled_y_train = pd.DataFrame(scaler_target.fit_transform(train_target), columns = ['energy_kwh'])  # 풍속 스케일링
scaled_y_test = pd.DataFrame(scaler_target.transform(test_target), columns = ['energy_kwh'])       # 테스트 데이터도 변환

scaled_y_train.set_index(train_yg.index, inplace=True)
scaled_y_test.set_index(test_yg_final.index, inplace=True)

# month, day, weekday
scaled_x_train['month'] = scaled_x_train.index.month
scaled_x_train['hour'] = scaled_x_train.index.hour

scaled_x_test['month'] = scaled_x_test.index.month
scaled_x_test['hour'] = scaled_x_test.index.hour

scaled_y_train['month'] = scaled_y_train.index.month
scaled_y_train['hour'] = scaled_y_train.index.hour

scaled_y_test['month'] = scaled_y_test.index.month
scaled_y_test['hour'] = scaled_y_test.index.hour


# 훈련 데이터 및 테스트 데이터 생성
X_train, y_train = create_features(feature = scaled_x_train, target = scaled_y_train, lookback = lookback)
X_test, y_test = create_features(feature = scaled_x_test, target = scaled_y_test, lookback = lookback)

# RandomForestRegressor 모델 훈련
rf_model = RandomForestRegressor(n_estimators=500, random_state=0)
rf_model.fit(X_train, y_train)

# 테스트 데이터에 대한 예측
pred_rf = rf_model.predict(X_test)

y_test = scaler_target.inverse_transform(y_test)
pred_rf = scaler_target.inverse_transform(pred_rf)


# 데이터프레임화
target_df = pd.DataFrame(pred_rf)

# 다음날 24시간 데이터만 뽑기
target_filtered = target_df.iloc[:, -24:]
target_reshaped = target_filtered.values.flatten()
target_transposed_pred = target_reshaped.transpose()

# 다시 한번 데이터 프레임화
yg_final = pd.DataFrame(target_transposed_pred, columns=['pred_energy'])

# 시간 컬럼 붙이기
yg_final['dt'] = test_yg_final.index[lookback+24:]

### (6) 제출 형식으로 변경

In [ ]:
# 제출형식으로 변경
pred_y.reset_index(inplace=True)
pred_y['plant_name'] = '영광풍력'
pred_y['period_hours'] = 1

pred_yg = pred_y[['plant_name', 'dt', 'period_hours', 'predicted_energy_kwh']]
pred_yg.columns = ['plant_name', 'end_datetime', 'period_hours', 'energy_kwh']

## 3-3 경주

### (1) test 데이터에 2022년 데이터 일부 붙이기
- lookback_days에 따라 2022년 12월 혹은 11월 데이터까지 사용해야할 수도 있음

In [ ]:
# 경주
test_gj_added = pd.concat([train_gj[train_gj.index >= '2022-11-01 00:00:00+09:00'], test_gj])
test_gj_added.index = pd.to_datetime(test_gj_added.index)

### (2) lookback_days 정의 후 lookback 계산

In [ ]:
# lookback_days : 며칠 전까지를 x시퀀스로 만들건지
lookback_days = 5

# 시간화
lookback = 24 * lookback_days

### (3) 컬럼 추가

In [ ]:
# 경주
gj_offset = test_gj_added[test_gj_added.index == '2022-12-31 00:00:00+09:00'].index[0] - pd.Timedelta(hours=lookback)
test_gj_final = test_gj_added[test_gj_added.index >= gj_offset]
test_gj_final['month'] = test_gj_final.index.month.astype('category')
test_gj_final['hour'] = test_gj_final.index.hour.astype('category')

### (4) 시퀀스 생성

In [ ]:
# 전날 15시까지의 데이터를 사용하여 다음날 23시까지 예측
def create_features(feature, target, lookback, forecast_horizon=32):
    X, y = [], []

    for i in range(lookback, len(feature) - forecast_horizon):
      current_time = feature.index[i]  # 현재 인덱스의 시간

      if current_time.hour == 15:  # 15시까지의 데이터를 사용하여 예측
          x_1 = feature.iloc[(i - lookback) : (i + 1)][['energy_kwh', 'wind_speed', 'wind_u_10m', 'storm_u_5m', 'temp_air', 'specific_humid']].values.flatten().reshape(1, -1)
          x_2 = np.array([feature[['month', 'hour']].values[i]])
          x_to = np.hstack([x_1, x_2])
          X.append(x_to.reshape(-1))  # 과거 (lookback)시간의 피처 데이터

          y_1 = feature.iloc[(i + 1) : (i + 1 + forecast_horizon)]['energy_kwh'].values
          y.append(y_1.flatten())  # 다음 32시간 동안의 예측 값

    return np.array(X), np.array(y)

### (5) 변수선택, 스케일링, 모델링

In [ ]:
gj_list = []

# 피처와 타겟 정의
train_features = train_gj[['energy_kwh', 'wind_speed', 'wind_u_10m', 'storm_u_5m', 'temp_air', 'specific_humid']]
train_target = train_gj[['energy_kwh']]

test_features = test_gj_final[['energy_kwh', 'wind_speed', 'wind_u_10m', 'storm_u_5m', 'temp_air', 'specific_humid']]
test_target = test_gj_final[['energy_kwh']]

# 데이터 스케일링
scaler_features = MinMaxScaler()
scaler_target = MinMaxScaler()

scaled_x_train = pd.DataFrame(scaler_features.fit_transform(train_features), columns = ['energy_kwh', 'wind_speed', 'wind_u_10m', 'storm_u_5m', 'temp_air', 'specific_humid'])  # train_features에 다차원 배열 입력
scaled_x_test = pd.DataFrame(scaler_features.transform(test_features) , columns = ['energy_kwh', 'wind_speed', 'wind_u_10m', 'storm_u_5m', 'temp_air', 'specific_humid'])       # test_features도 동일한 방식으로 변환

scaled_x_train.set_index(train_gj.index, inplace=True)
scaled_x_test.set_index(test_gj_final.index, inplace=True)

# 풍속 데이터에 대한 스케일링
scaled_y_train = pd.DataFrame(scaler_target.fit_transform(train_target), columns = ['energy_kwh'])  # 풍속 스케일링
scaled_y_test = pd.DataFrame(scaler_target.transform(test_target), columns = ['energy_kwh'])       # 테스트 데이터도 변환

scaled_y_train.set_index(train_gj.index, inplace=True)
scaled_y_test.set_index(test_gj_final.index, inplace=True)

# month, day, weekday
scaled_x_train['month'] = scaled_x_train.index.month
scaled_x_train['hour'] = scaled_x_train.index.hour

scaled_x_test['month'] = scaled_x_test.index.month
scaled_x_test['hour'] = scaled_x_test.index.hour

scaled_y_train['month'] = scaled_y_train.index.month
scaled_y_train['hour'] = scaled_y_train.index.hour

scaled_y_test['month'] = scaled_y_test.index.month
scaled_y_test['hour'] = scaled_y_test.index.hour


# 훈련 데이터 및 테스트 데이터 생성
X_train, y_train = create_features(feature = scaled_x_train, target = scaled_y_train, lookback = lookback)
X_test, y_test = create_features(feature = scaled_x_test, target = scaled_y_test, lookback = lookback)

# RandomForestRegressor 모델 훈련
rf_model = RandomForestRegressor(n_estimators=500, random_state=0)
rf_model.fit(X_train, y_train)

# 테스트 데이터에 대한 예측
pred_rf = rf_model.predict(X_test)

y_test = scaler_target.inverse_transform(y_test)
pred_rf = scaler_target.inverse_transform(pred_rf)

# 데이터프레임화
target_df = pd.DataFrame(pred_rf)

# 다음날 24시간 데이터만 뽑기
target_filtered = target_df.iloc[:, -24:]
target_reshaped = target_filtered.values.flatten()
target_transposed_pred = target_reshaped.transpose()

# 다시 한번 데이터 프레임화
gj_final = pd.DataFrame(target_transposed_pred, columns=['pred_energy'])

# 시간 컬럼 붙이기
gj_final['dt'] = test_gj_final.index[lookback+24:]

### (6) 제출 형식으로 변경

In [ ]:
# 제출형식으로 변경
pred_y.reset_index(inplace=True)
pred_y['plant_name'] = '경주풍력'
pred_y['period_hours'] = 1

pred_gj = pred_y[['plant_name', 'dt', 'period_hours', 'predicted_energy_kwh']]
pred_gj.columns = ['plant_name', 'end_datetime', 'period_hours', 'energy_kwh']

## 4. 예측 데이터 저장

In [ ]:
pred = pd.concat([pred_yg, pred_gj])

In [ ]:
pred.to_csV(os.path.join('WINDmin_result.csv'), encoding='cp949',index=False)